In [1]:
import vtk
from ipyvtklink.viewer import ViewInteractiveWidget
import numpy as np
import ipywidgets as widgets
from IPython.display import display

In [2]:
hip_path = "/home/mahsadibaji/mdsc689-03-notebooks/Hip"
thrx_path = "/home/mahsadibaji/mdsc689-03-notebooks/Thorax"
head_path = "/home/mahsadibaji/mdsc689-03-notebooks/head.nii"


In [14]:
class vtk_pipeline:
    
    def __init__(self, path, filetype, render_size = 300):
        self.path = path
        self.filetype = filetype
        self.render_size = render_size
        

        self.out, self.outport, self.dim = self.image_reader()
                
        self.resizers = {}
        self.mappers = {}
        self.actors = {}
        
        self.kernel_size = (3, 3, 3)
        
        self.segmentation_color = (1.0, 0.0, 0.0) #RGB value
        self.segmentation_opacity = 1.0
        self.segmented = None
        
        self.renderer = None
        self.render_window = None
        self.interactor = None
        
    def image_reader(self):   
        if self.filetype.lower() == 'dicom':  # Load a DICOM image from file into a vtk pipeline.
            img_reader = vtk.vtkDICOMImageReader()
            img_reader.SetDirectoryName(self.path)
            img_reader.Update()
            out = img_reader.GetOutput()
            outport = img_reader.GetOutputPort()
            dim = out.GetDimensions()
            
        elif self.filetype.lower() == 'niftii': # Load a NIFTI image from file into a vtk pipeline.
            img_reader = vtk.vtkNIFTIImageReader()
            img_reader.SetFileName(self.path)
            img_reader.Update()
            out = img_reader.GetOutput()
            outport = img_reader.GetOutputPort()
            dim = out.GetDimensions()
                    
        print("image read")
        
        return out, outport, dim
    
                        
    def gaussian_filter(self): #Done
        dimensionality = 3
        
        gaussian = vtk.vtkImageGaussianSmooth()
        gaussian.SetDimensionality(dimensionality)
        gaussian.SetInputData(self.out)
        gaussian.Update()
        
        return gaussian

    def render_surface(self, iso_value, color=None, g_filter=True, method="Marching"):
        
        if g_filter:
            self.out = self.gaussian_filter().GetOutput()
            
        if method.lower()=="marching":
            print("MarchingCubes")
            surface = vtk.vtkImageMarchingCubes()
            surface.SetInputData(self.out)
            surface.SetValue(0, iso_value)
            surface.ComputeNormalsOn()
            surface.Update()
            
        elif method.lower()=="contour":
            print("ContourFilter")
            surface = vtk.vtkContourFilter()
            surface.SetInputData(self.out)
            surface.SetValue(0, iso_value)
            surface.Update()
        else:
            print("invalid method choice")
            
        lut = vtk.vtkLookupTable() # Lookup table to set the color of 3D surface rendering
        lut.SetNumberOfTableValues(2)
        lut.SetTableValue(0, 1, 0, 0, 1)
        lut.SetTableValue(1, *color, 1) 

        surface_mapper = vtk.vtkPolyDataMapper()
        surface_mapper.SetInputData(surface.GetOutput())
        surface_mapper.SetLookupTable(lut) 
        surface_mapper.Update()
    
        
        surface_actor = vtk.vtkActor()
        surface_actor.SetMapper(surface_mapper)

        self.render_window = vtk.vtkRenderWindow()
        self.render_window.SetOffScreenRendering(True)
        self.render_window.SetSize(self.render_size, self.render_size)

        self.renderer = vtk.vtkRenderer()
        self.renderer.ResetCamera()       
        self.render_window.AddRenderer(self.renderer)
        
        self.renderer.AddActor(surface_actor)

        interactorStyle = vtk.vtkInteractorStyleTrackballActor()

        self.interactor = vtk.vtkRenderWindowInteractor()
        self.interactor.SetRenderWindow(self.render_window)
        self.interactor.SetInteractorStyle(interactorStyle)        
        
        self.interactor.Initialize()
        
    def render_volume(self, transfer_colors=None, transfer_op = None, g_filter=True):
        
        if g_filter:
            self.out = self.gaussian_filter().GetOutput()            
        print('image mapper created')
        
        ctf = vtk.vtkColorTransferFunction()  #Set different colors for different organs based on intensity values
        for val in transfer_colors.values():  
            ctf.AddRGBPoint(*val)        
        
        otf = vtk.vtkPiecewiseFunction() #Set different opacity for different organs based on intensity values
        for val in transfer_op.values():
            otf.AddPoint(*val)
        
        #Set volume properties      
        volume_property = vtk.vtkVolumeProperty()
        volume_property.ShadeOn()
        volume_property.SetScalarOpacity(otf)
        volume_property.SetColor(ctf)
        volume_property.SetInterpolationTypeToLinear()
        
        volume_mapper = vtk.vtkFixedPointVolumeRayCastMapper()
        volume_mapper.SetInputData(self.out)
        volume_mapper.Update()
        
        volume_actor = vtk.vtkVolume()
        volume_actor.SetMapper(volume_mapper)
        volume_actor.SetProperty(volume_property)

        self.renderer = vtk.vtkRenderer()
        self.renderer.AddVolume(volume_actor)
        self.renderer.ResetCamera()

        self.render_window = vtk.vtkRenderWindow()
        self.render_window.SetOffScreenRendering(True)
        self.render_window.SetSize(self.render_size, self.render_size)
        self.render_window.AddRenderer(self.renderer)

        interactorStyle = vtk.vtkInteractorStyleTrackballActor()

        self.interactor = vtk.vtkRenderWindowInteractor()
        self.interactor.SetRenderWindow(self.render_window)
        self.interactor.SetInteractorStyle(interactorStyle)        

        self.interactor.Initialize()        

# Experiments

## 3D Surface Rendering
Two methods are used to render surfaces for each sample dataset. The results are almost similar.

1. `vtkImageMarchingCubes()`: Marching Cubes is a 3D isosurface extraction algorithm that works by dividing a 3D space into smaller cubes and extracting an isosurface by examining the values at the vertices of each cube and mapping the resulting configuration to a set of triangles that define the surface. This process is repeated for all cubes to create the final isosurface. Marching Cubes is widely used for extracting surfaces from scalar data in scientific visualization and medical imaging.
2. `.vtkContourFilter()`: This algorithm generates isosurfaces from structured or unstructured point data. It can be used to extract surfaces from volumetric data, or to generate contours from scalar or vector data.

In [49]:
head_m = vtk_pipeline(path=head_path, filetype='niftii', render_size=600)

head_m.render_surface(iso_value=500, color=(0.9, 0.3, 0.5),g_filter=True, method="marching")

ViewInteractiveWidget(head_m.render_window)

image read


ViewInteractiveWidget(height=600, layout=Layout(height='auto', width='100%'), width=600)

Brain Surface rendering(head dataset) after Gaussian filtering using MarchingCubes Algorithm: iso_value=500

In [55]:
head_c = vtk_pipeline(path=head_path, filetype='niftii', render_size=600)

head_c.render_surface(iso_value=500, color=(0.9, 0.3, 0.5),g_filter=True, method="contour")

ViewInteractiveWidget(head_c.render_window)

image read


ViewInteractiveWidget(height=600, layout=Layout(height='auto', width='100%'), width=600)

Brain Surface rendering (head dataset) after Gaussian filtering using ContourFilter Algorithm: iso_value=500

In [51]:
thorax_m = vtk_pipeline(path=thrx_path, filetype='dicom', render_size=500)
thorax_m.render_surface(iso_value=200, color=(1, 0.8, 0.7), g_filter=True, method="marching")

ViewInteractiveWidget(thorax_m.render_window)

image read


ViewInteractiveWidget(layout=Layout(height='auto', width='100%'), width=500)

Thorax Surface rendering after Gaussian filtering using MarchingCubes Algorithm: iso_value=200

In [58]:
thorax_c = vtk_pipeline(path=thrx_path, filetype='dicom', render_size=500)
thorax_c.render_surface(iso_value=200, color=(1, 0.8, 0.7), g_filter=True, method="contour")

ViewInteractiveWidget(thorax_c.render_window)

image read
ContourFilter


ViewInteractiveWidget(layout=Layout(height='auto', width='100%'), width=500)

Thorax Surface rendering after Gaussian filtering using ContourFilter Algorithm: iso_value=200

In [15]:
hip_m = vtk_pipeline(path=hip_path, filetype='dicom', render_size=500)
hip_m.render_surface(iso_value=200,color=(1, 0.8, 0.7), g_filter=True, method="marching")

ViewInteractiveWidget(hip_m.render_window)

image read
MarchingCubes


ViewInteractiveWidget(layout=Layout(height='auto', width='100%'), width=500)

Hip Surface rendering after Gaussian filtering using MarchingCubes Algorithm: iso_value=200

In [59]:
hip_c = vtk_pipeline(path=hip_path, filetype='dicom', render_size=500)
hip_c.render_surface(iso_value=200,color=(1, 0.8, 0.7), g_filter=True, method="contour")

ViewInteractiveWidget(hip_c.render_window)

image read
ContourFilter


ViewInteractiveWidget(layout=Layout(height='auto', width='100%'), width=500)

Hip Surface rendering after Gaussian filtering using Contour Algorithm: iso_value=200

## Volume Rendering

In [72]:
head_vol = vtk_pipeline(path=head_path, filetype='niftii', render_size=600)

head_tc = {
    "back_ground": [0, 0.0, 0.0, 0.0],
    "skin": [400,1, 0.8, 0.7],
    "bone": [450, 0.9, 0.3, 0.5]
}


head_to = {
    "back_ground": [0,0.0],
    "skin": [400,0.03],
    "bone": [450,1]
}

head_vol.render_volume(transfer_colors=head_tc , transfer_op=head_to, g_filter=True)

ViewInteractiveWidget(head_vol.render_window)

image read
image mapper created


ViewInteractiveWidget(height=600, layout=Layout(height='auto', width='100%'), width=600)

Head Volume Rendering after Gaussian filtering: different colors are being used for the organs inside head (pink) with intensity 450 and skin/scalp(skin color) with intensity 400

In [10]:
thrx_vol = vtk_pipeline(path=thrx_path, filetype='dicom', render_size=600)

thrx_tc = {
    "back_ground": [0, 0.0, 0.0, 0.0],
    "skin": [80,1, 0.8, 0.7],
    "bone": [350, 0.98, 0.96, 0.93]
}

thrx_to = {
    "back_ground": [0,0.0],
    "skin": [80,0.02],
    "bone": [350,1]
}


thrx_vol.render_volume(transfer_colors=thrx_tc , transfer_op=thrx_to, g_filter=True)

ViewInteractiveWidget(thrx_vol.render_window)

image read
image mapper created


ViewInteractiveWidget(height=600, layout=Layout(height='auto', width='100%'), width=600)

Thorax Volume Rendering after Gaussian filtering: different colors are being used for the bone(bone color) with intensity 350 and skin/other organs(skin color) with intensity 80

In [8]:
hip_vol = vtk_pipeline(path=hip_path, filetype='dicom', render_size=600)

hip_tc = {
    "back_ground": [0, 0.0, 0.0, 0.0],
    "skin": [80,1, 0.8, 0.7],
    "bone": [350, 0.98, 0.96, 0.93]
}

hip_to = {
    "back_ground": [0,0.0],
    "skin": [80,0.03],
    "bone": [350,1]
}


hip_vol.render_volume(transfer_colors=hip_tc , transfer_op=hip_to, g_filter=True)

ViewInteractiveWidget(hip_vol.render_window)

image read
image mapper created


ViewInteractiveWidget(height=600, layout=Layout(height='auto', width='100%'), width=600)

Hip Volume Rendering after Gaussian filtering: different colors are being used for the bone (bone color) with intensity 350 and skin/other organs(skin color) with intensity 80

# User Input

In [13]:
image_options = [ ('Head', head_path), ('Hip', hip_path), ('Thorax', thrx_path)]
image = widgets.Dropdown(
    options=image_options,
    description='Image:',
)
filetype = widgets.Dropdown(
        options = ['dicom','niftii'],
        value = 'niftii',
        description = 'File Type: '
)
rendering = widgets.Dropdown(
        options = ['surface','volume'],
        value = 'volume',
        description = 'Rendering Type: '
)
method = widgets.Dropdown(
        options = ['marching','contour','None'],
        value = 'None',
        description = 'Surface Method: '
)

iso_value = widgets.IntText(description="iso_value:", value=0)

tofilter = widgets.Checkbox(
    value=True,
    description='apply Gaussian filter',
)


thresh1 = widgets.IntText(description="intenisty 1:", value=100)
thresh2 = widgets.IntText(description="intensity 2:", value=300)

display(image, filetype, rendering, method, iso_value, thresh1, thresh2, tofilter)

Dropdown(description='Image:', options=(('Head', '/home/mahsadibaji/mdsc689-03-notebooks/head.nii'), ('Hip', '…

Dropdown(description='File Type: ', index=1, options=('dicom', 'niftii'), value='niftii')

Dropdown(description='Rendering Type: ', index=1, options=('surface', 'volume'), value='volume')

Dropdown(description='Surface Method: ', index=2, options=('marching', 'contour', 'None'), value='None')

IntText(value=0, description='iso_value:')

IntText(value=100, description='intenisty 1:')

IntText(value=300, description='intensity 2:')

Checkbox(value=True, description='apply Gaussian filter')

In [16]:
obj = vtk_pipeline(path=head_path, filetype='niftii', render_size=600)
if rendering.value == "surface":
    color=(1, 0.8, 0.7)
    obj.render_surface(iso_value=iso_value.value,color=color, g_filter=tofilter.value, method=method.value)
    
elif rendering.value == "volume":
    tc = {
        "back_ground": [0, 0.0, 0.0, 0.0],
        "skin": [thresh1.value,1, 0.8, 0.7],
        "bone": [thresh2.value, 0.98, 0.96, 0.93]
    }

    to = {
        "back_ground": [0,0.0],
        "skin": [thresh1.value,0.03],
        "bone": [thresh2.value,1]
    }


    obj.render_volume(transfer_colors=tc , transfer_op=to, g_filter=tofilter.value)

ViewInteractiveWidget(obj.render_window)

image read
MarchingCubes


ViewInteractiveWidget(height=600, layout=Layout(height='auto', width='100%'), width=600)